<a href="https://colab.research.google.com/github/T-Sawao/diveintocode-ml3/blob/main/term2_sprint11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2.1次元の畳み込みニューラルネットワークスクラッチ

畳み込みニューラルネットワーク（CNN） のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。


このSprintでは1次元の 畳み込み層 を作成し、畳み込みの基礎を理解することを目指します。次のSprintでは2次元畳み込み層とプーリング層を作成することで、一般的に画像に対して利用されるCNNを完成させます。


クラスの名前はScratch1dCNNClassifierとしてください。クラスの構造などは前のSprintで作成したScratchDeepNeuralNetrowkClassifierを参考にしてください。


**1次元畳み込み層とは**  
CNNでは画像に対しての2次元畳み込み層が定番ですが、ここでは理解しやすくするためにまずは1次元畳み込み層を実装します。1次元畳み込みは実用上は自然言語や波形データなどの 系列データ で使われることが多いです。


畳み込みは任意の次元に対して考えることができ、立体データに対しての3次元畳み込みまではフレームワークで一般的に用意されています。


**データセットの用意**  
検証には引き続きMNISTデータセットを使用します。1次元畳み込みでは全結合のニューラルネットワークと同様に平滑化されたものを入力します。

## 【問題1】チャンネル数を1に限定した1次元畳み込み層クラスの作成
チャンネル数を1に限定した1次元畳み込み層のクラスSimpleConv1dを作成してください。基本構造は前のSprintで作成した全結合層のFCクラスと同じになります。なお、重みの初期化に関するクラスは必要に応じて作り変えてください。Xavierの初期値などを使う点は全結合層と同様です。

ここでは パディング は考えず、ストライド も1に固定します。また、複数のデータを同時に処理することも考えなくて良く、バッチサイズは1のみに対応してください。この部分の拡張はアドバンス課題とします。

フォワードプロパゲーションの数式は以下のようになります。
$$\alpha_i = \sum_{s=0}^{F-1}x_{(i+s)}W_s + b$$

$a_i$ : 出力される配列のi番目の値

$F$ : フィルタのサイズ

$x_{(i+s})$ : 入力の配列の(i+s)番目の値

$w_s$ : 重みの配列のs番目の値

$b$ : バイアス項

全てスカラーです。

次に更新式です。ここがAdaGradなどに置き換えられる点は全結合層と同様です。
$$w'_s = w_s -\alpha\frac{\partial L}{\partial w_s}$$$$b' = b - \alpha\frac{\partial L}{\partial b}$$

$\alpha$ : 学習率

$\frac{\partial L}{\partial w_s}$ : w_s に関する損失 $L$ の勾配

$\frac{\partial L}{\partial b}$ : $b$ に関する損失 $L$ の勾配

勾配 $\frac{\partial L}{\partial w_s}$ や $\frac{\partial L}{\partial b}$ を求めるためのバックプロパゲーションの数式が以下です。

$$\frac{\partial L}{\partial w_s} = \sum_{i=0}^{N_{out}-1}\frac{\partial L}{\partial a_i}x_{(i+s}$$$$\frac{\partial L}{\partial b} =  \sum_{i=0}^{N_{out}-1}\frac{\partial L}{\partial a_i}$$
$\frac{\partial L}{\partial a_i}$: 勾配の配列のi番目の値

$N_{out} : 出力のサイズ

前の層に流す誤差の数式は以下です。　　$$\frac{\partial L}{\partial x_j} = \sum_{s=0}^{F-1}\frac{\partial L}{\partial a_{j-s}}w_s$$

$\frac{\partial L}{\partial x_j}$ : 前の層に流す誤差の配列のj番目の値

ただし、 $j−s&lt;0$ または$j−s&gt;N_{out}−1$ のとき$\frac{\partial L}{\partial a_{j-s}}=0$ です。

全結合層との大きな違いは、重みが複数の特徴量に対して共有されていることです。この場合は共有されている分の誤差を全て足すことで勾配を求めます。計算グラフ上での分岐はバックプロパゲーションの際に誤差の足し算をすれば良いことになります。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as setattr
from sklearn.metrics import accuracy_score

In [ ]:
x = np.array([1,2,3,4])
y = np.array([45, 70])
w = np.array([3, 5, 7])
b = np.array([1])

F = w.shape[0]
P = 0
S = 1

### 1.1.1（解答）フォワードプロパゲーション

In [ ]:
y_hat = np.zeros(F-1)
for i in range(F-1):
  count = 0
  for s in range(w.shape[0]):
    count += x[i+s] * w[s]
  y_hat[i] = count + b
y_hat

1.1.2（別解)

In [ ]:
y_hat1 = np.zeros(F-1)
for m in range(F-1):
  y_hat1[m] = np.sum((x[m:m+F])*w)+b
print(y_hat1)

### 1.2.1（解答）更新式

In [ ]:
loss = y - y_hat
loss

### 1.3.1（解答）バックプロパゲーション

In [ ]:
db = np.sum(loss)
db

In [ ]:
dw = np.zeros(w.shape[0])
for k in range(2):
  dw += loss[k] * x[k:k+F]
print("dw", dw)

In [ ]:
dx = np.sum([np.r_[0, loss[1]*w], np.r_[loss[0]*w, 0]], axis=0)
dx

## 【問題2】1次元畳み込み後の出力サイズの計算¶
畳み込みを行うと特徴量の数が変化します。どのように変化するかは以下の数式から求められます。パディングやストライドも含めています。この計算を行う関数を作成してください。
$$N_{out} = \frac{N_{in} + 2P - F}{S} + 1$$

$N_{out}$ : 出力のサイズ（特徴量の数）

$N_{in}$ : 入力のサイズ（特徴量の数）

$P$ : ある方向へのパディングの数

$F$ : フィルタのサイズ

$S$ : ストライドのサイズ

### 2.1.1（解答）

In [ ]:
Nout = ((len(x) + 2*P - F)/S)+1
Nout

## 【問題3】小さな配列での1次元畳み込み層の実験
次に示す小さな配列でフォワードプロパゲーションとバックプロパゲーションが正しく行えているか確認してください。

入力$x$、重み$w$、バイアス$b$を次のようにします。

- x = np.array([1,2,3,4])
- w = np.array([3, 5, 7])
- b = np.array([1])  

フォワードプロパゲーションをすると出力は次のようになります。

- a = np.array([35, 50])

次にバックプロパゲーションを考えます。誤差は次のようであったとします。

-delta_a = np.array([10, 20])

バックプロパゲーションをすると次のような値になります。

- delta_b = np.array([30])
- delta_w = np.array([50, 80, 110])
- delta_x = np.array([30, 110, 170, 140])

**実装上の工夫**  
畳み込みを実装する場合は、まずはfor文を重ねていく形で構いません。しかし、できるだけ計算は効率化させたいため、以下の式を一度に計算する方法を考えることにします。
$$a_i = \sum_{s=0}^{F-1}x_{i+s}w_s + b$$

バイアス項は単純な足し算のため、重みの部分を見ます。$$\sum_{s=0}^{F-1}x_{i+s}w_s$$

これは、xの一部を取り出した配列とwの配列の内積です。具体的な状況を考えると、以下のようなコードで計算できます。この例では流れを分かりやすくするために、各要素同士でアダマール積を計算してから合計を計算しています。これは結果的に内積と同様です。

- x = np.array([1, 2, 3, 4])
- w = np.array([3, 5, 7])
- a = np.empty((2, 3))
- indexes0 = np.array([0, 1, 2]).astype(np.int)
- indexes1 = np.array([1, 2, 3]).astype(np.int)
- a[0] = x[indexes0]w # x[indexes0]は([1, 2, 3])である 
- a[1] = x[indexes1]w # x[indexes1]は([2, 3, 4])である
- a = a.sum(axis=1)

ndarrayは配列を使ったインデックス指定ができることを利用した方法です。

また、二次元配列を使えば一次元配列から二次元配列が取り出せます。

- x = np.array([1, 2, 3, 4])
- indexes = np.array([[0, 1, 2], [1, 2, 3]]).astype(np.int)
- print(x[indexes]) # ([[1, 2, 3], [2, 3, 4]])

このこととブロードキャストなどをうまく組み合わせることで、一度にまとめて計算することも可能です。

畳み込みの計算方法に正解はないので、自分なりに効率化していってください。

**《参考》**  
以下のページのInteger array indexingの部分がこの方法についての記述です。

Indexing — NumPy v1.17 Manual  
https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html

## 3.1.1(解答)

In [ ]:
class SimpleConv1d:
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        # self.W = initializer.W(self.n_nodes1, self.n_nodes2)
        # self.B = initializer.B(self.n_nodes2)
        self.w = np.array([3, 5, 7])
        self.b = np.array([1])
        self.P = 0
        self.S = 1
        self.F = len(self.w)

        self.dw = np.array([])
        self.dx = np.array([])

# 問題1--------------------------------------------------------------
    # フォワードプロパゲーション時の処理
    def forward(self, x):
      for m in range(self.F-1):
        y_hat1[m] = np.sum((x[m:m+self.F])*self.w)+self.b
      return y_hat1

    # バックプロパゲーション時の処理
    def backward(self, X, da):
      db = np.sum(da)
      dw = np.zeros(self.w.shape[0])
      for i in range(int(self._output_size(X))):
        dw += da[i] * x[i:i+self.F]

      dx = np.sum([np.r_[0, da[1]*self.w], np.r_[da[0]*self.w, 0]], axis=0)
      return db, dw, dx

# 問題2--------------------------------------------------------------
    def _output_size(self, X):
      self.Nout = int((len(X)) + (2*self.P) - self.F) / self.S + 1
      return self.Nout

In [ ]:
test = SimpleConv1d(1,2,3,4)
test.forward(x)

In [ ]:
test._output_size(x)

In [ ]:
db, dw, dx = test.backward(x, loss)
print("db", db, "dw", dw, "dx",dx)

## 【問題4】チャンネル数を限定しない1次元畳み込み層クラスの作成
チャンネル数を1に限定しない1次元畳み込み層のクラスConv1dを作成してください。

例えば以下のようなx, w, bがあった場合は、

- x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
- w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。
- b = np.array([1, 2, 3]) # （出力チャンネル数）

出力は次のようになります。

- a = np.array([[16, 22], [17, 23], [18, 24]]) # shape(3, 2)で、（出力チャンネル数、特徴量数）である。

入力が2チャンネル、出力が3チャンネルの例です。計算グラフを書いた上で、バックプロパゲーションも手計算で考えてみましょう。計算グラフの中には和と積しか登場しないので、微分を新たに考える必要はありません。

**《補足》**  
チャンネル数を加える場合、配列をどういう順番にするかという問題があります。(バッチサイズ、チャンネル数、特徴量数)または(バッチサイズ、特徴量数、チャンネル数)が一般的で、ライブラリによって順番は異なっています。（切り替えて使用できるものもあります）

今回のスクラッチでは自身の実装上どちらが効率的かを考えて選んでください。上記の例ではバッチサイズは考えておらず、(チャンネル数、特徴量数)です。

### 4.1.1（解答）

In [ ]:
class SimpleConv1_1d:
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        # self.W = initializer.W(self.n_nodes1, self.n_nodes2)
        # self.B = initializer.B(self.n_nodes2)

        # diver確認用サンプルデータ
        self.W = np.array([[[1,1,1],[1,1,1]],[[1,1,1],[2,1,1]],[[2,1,1],[1,1,2]]])
        self.b =np.array([3, 2, 1]) 

        self.P = 0 # パディング(今回未使用)
        self.S = 1 # ストライド値
        self.F = int(self.W.shape[2])  # フィルターサイズ

        self.dw = np.array([])
        self.dx = np.array([])

# 問題1--------------------------------------------------------------
    # フォワードプロパゲーション時の処理
    def forward(self, x):
      self.X = x
      self.Nout = self._output_size(self.X)

      # 出力値を記載する枠を作成
      a = np.zeros([self.W.shape[0], self.Nout])
      for j in range(self.F):
        for i in range(self.Nout):
          # 1.フィルターを移動させ、w重みを掛け合わす。
          # 2.上記作業をチャンネル回数繰り返す。
          a[j,i] = np.sum(self.X[:, i:i+self.F]*self.W[j])+self.b[j]
          #                       -----------
          #                       列の指定（フィルター)[i]から[i+F]までの値を返す。
      return a

      # 更新
      self = self.optimizer.updatbe(self)
      return self

    # バックプロパゲーション時の処理
    def backward(self, loss):
      db = np.sum(loss)
      dw = np.zeros(self.W.shape)
      dx = np.zeros(self.X.shape)
      for j in range(self.W.shape[0]):
        for i in range(self.W.shape[1]):

          # dwの式　W.shape(2,3)を１つとして、1つ目のfor分で行軸をスライドさせ、２つ目のfor分でチャネルを更新。
          dw[j] += loss[j,i] * self.X[:,i:i+3]

        # dxの式 W.shape(1,3)を１つとして、loss[0,0]*W[0]+loss[0,1]*W[0]と、
        # loss[0,0]*W[1]+loss[0,1]*W[1]でfor文でチャネルを更新して、各要素ごとに足して合計を返す。
        dx += np.r_[[np.r_[np.sum([loss[i,0]*self.W[i,0,:]], axis=0), 0] + np.r_[0,np.sum([loss[i,1]*self.W[i,0,:]], axis=0)]],
                     [np.r_[np.sum([loss[i,0]*self.W[i,1,:]], axis=0), 0] + np.r_[0,np.sum([loss[i,1]*self.W[i,1,:]], axis=0)]]]
      return db, dw, dx

# 問題2--------------------------------------------------------------
    def _output_size(self, X):
      Nout = int(((X.shape[1]) + (2*self.P) - self.F) / self.S + 1)
      return Nout

In [ ]:
X = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
W = np.array([[[1,1,1],[1,1,1]],[[1,1,1],[2,1,1]],[[2,1,1],[1,1,2]]])
B = np.array([1, 2, 3]) # （出力チャンネル数）
loss = np.array([[52,56],[32,35],[9,11]])

In [ ]:
test2 = SimpleConv1_1d(1,2,3,4)
test2.forward(X)

### 4.2.1 (解答）バックプロパゲーション

In [ ]:
db, dw, dx = test2.backward(loss)
print("db",db)
print("dw",dw)
print("dx", dx)

In [ ]:
dw = np.zeros(W.shape)
for j in range(3):
  for i in range(2):
    dw[j] += loss[j,i] * X[:,i:i+3]
dw

In [ ]:
x = np.zeros(X.shape)
for i in range(3):
  x += np.r_[[np.r_[np.sum([loss[i,0]*W[i,0,:]], axis=0), 0] + np.r_[0,np.sum([loss[i,1]*W[i,0,:]], axis=0)]],
             [np.r_[np.sum([loss[i,0]*W[i,1,:]], axis=0), 0] + np.r_[0,np.sum([loss[i,1]*W[i,1,:]], axis=0)]]]
x

## 【問題5】（アドバンス課題）パディングの実装
畳み込み層にパディングの機能を加えてください。1次元配列の場合、前後にn個特徴量を増やせるようにしてください。


最も単純なパディングは全て0で埋める ゼロパディング であり、CNNでは一般的です。他に端の値を繰り返す方法などもあります。


フレームワークによっては、元の入力のサイズを保つようにという指定をすることができます。この機能も持たせておくと便利です。なお、NumPyにはパディングの関数が存在します。


numpy.pad — NumPy v1.17 Manual  
https://docs.scipy.org/doc/numpy/reference/generated/numpy.pad.html



## 【問題6】（アドバンス課題）ミニバッチへの対応
ここまでの課題はバッチサイズ1で良いとしてきました。しかし、実際は全結合層同様にミニバッチ学習が行われます。Conv1dクラスを複数のデータが同時に計算できるように変更してください。

## 【問題7】（アドバンス課題）任意のストライド数
ストライドは1限定の実装をしてきましたが、任意のストライド数に対応できるようにしてください。

# 3.検証

##【問題8】学習と推定  
これまで使ってきたニューラルネットワークの全結合層の一部をConv1dに置き換えてMNISTを学習・推定し、Accuracyを計算してください。


出力層だけは全結合層をそのまま使ってください。ただし、チャンネルが複数ある状態では全結合層への入力は行えません。その段階でのチャンネルは1になるようにするか、 平滑化 を行なってください。


画像に対しての1次元畳み込みは実用上は行わないことのため、精度は問いません。

In [ ]:
# メインクラス
class ScratchDeepNeuralNetworkClassifier():
  def __init__(self, filter_num, lr=0.01, sigma=0.01, epoch_num=1, batch_size=20, n_output=10): 
    self.Fn = filter_num
    self.lr = lr
    self.sigma = sigma
    self.epoch = epoch_num
    self.batch_size = batch_size
    self.n_output = n_output
    
  def fit(self, X, y, ln_ch, initializing, activations, filter=3, stride=1, pad=0, opt="SGD"):
    self.n_features = X.shape[-1]
    self.loss = np.zeros(epoch)
    self.activation_list = []
    sefl.FC_list = []
    self.conv_list = []
    self.F = filter
    self.S = stride
    self.P = pad
  
    n_node1 = self.n_features
    self.flat_out_ch, self.flat_out_w = 0, 0
    
    # インスタンス生成----------------------------------------
    for i in range(len(initializing)):

      # 初期値-------------------------------------------
      if initializing[i] == "XavierInitializer":
        initializing = XavierInitializer(self.sigma)
      elif initializing[i] == "HeInitializer":
        initializing = HeInitializer(self.sigma)
      else:
        initializing = SimpleInitializer(self.sigma)

      # 最適化-------------------------------------------
      if opt == "AdaGrad":
        optimizer = AdaGrad(self.lr)
      else:
        optimizer = SGD(self.lr)

      # layer -------------------------------------------
      if i < len(self.Fn):
        n_node2 = self._output_size(n_node1)
        self.conv_list.append(Conv1_1d(In_ch, self.Fn[i], n_node1, n_node2, filter, stride, pad, initializing, optimizer))
      else:
        self.FC_list.append(FC(n_node1, self.n_output, initializing, optimizer))
          
      if i == len(self.Fn)-1:
        self.flat_out_ch, self.flat_out_w = self.Fn[-1], n_node2
        n_node1 = self.Fn[-1]*n_node2
      else:
        n_node1 = n_node2
      
      # 活性化関数インスタンス生成----------------
      if i < len(self.Fn):
        if activations[i] == "ReLU":
          self.activation_list.append(ReLU())
        elif activations[i] == "sigmoid":
          self.activation_list.append(Sigmoid())
        else:
          self.activation_list.append(Tanh())
      else:
        self.activation_list.append(Softmax())
      # -----------------------------------------------------------------end

    # エポック数分の学習
    for i in range(self.epoch):
      A = self.conv_list[0].forward(X)    # 畳み込み層 1層目
      Z = self.activation_list[0].forward(A)
      
      for j in range(1, len(self.conv_list) -1):    # 畳み込み層 2層目以降
          A = self.conv_list[j].forward(Z)
          Z = self.activation_list[j].forward(A)
      
      # conv→FCのreshape バッチサイズ=1なので1
      Z = Z.reshape(1, self.flat_Out_ch*self.flat_Out_w)
      
      A = self.FC_list[-1].forward(Z)    # 出力層
      Z = self.activation_list[-1].forward(A)
      
      dA = self.activation_list[-1].backward(y) # 出力層
      dZ = self.FC_list[-1].backward(dA)
      # FC→convのreshape
      dZ = dZ.reshape(self.flat_Out_ch, self.flat_Out_w)
      
      for j in reversed(range(1, len(self.conv_list))):    # 中間層
          dA = self.activation_list[j].backward(dZ)
          dZ = self.conv_list[j].backward(dA)

      dA = self.activation_list[0].backward(dZ)    # 入力層
      dZ = self.conv_list[0].backward(dA)
          
          
      yp = self.predict(X)
      # 損失関数
      self.loss[i] = -np.sum(y * np.log(self.predict_y))/ self.predict_y.shape[1]


  def predict(self, X):
    A = self.conv_list[0].forward(X)    # 入力層
    Z = self.activation_list[0].forward(A)
    
    for j in range(1, len(self.conv_list)-1):    # 中間層&出力層
        A = self.conv_list[j].forward(Z)
        Z = self.activation_list[j].forward(A)
        
    # conv→FCのreshape バッチサイズ=1なので1
    Z = Z.reshape(1, self.flat_Out_ch*self.flat_Out_w)
    
    A = self.FC_list[-1].forward(Z)
    Z = self.activation_list[-1].forward(A)
    self.predict_y = Z.copy()
    return np.argsort(Z)[:,-1]

  def _output_size(self, X):
    Nout = int(((X.shape[1]) + (2*self.P) - self.F) / self.S + 1)
    return Nout

In [ ]:
# チャンネル数１の畳み込みネットワーク
class Conv1_1d:
    def __init__(self, n_nodes1, n_nodes2, filter, stride, pad, initializer, optimizer):
        self.optimizer = optimizer
        self.P = pad    # パディング(今回未使用)
        self.S = stride  # ストライド値
        self.F = filter   # フィルターサイズ
        self.W = initializer.W(n_nodes1, n_nodes2, self.F)
        self.B = initializer.B(n_nodes2)

# 問題1--------------------------------------------------------------
    # フォワードプロパゲーション時の処理
    def forward(self, x):
      self.X = x
      self.Nout = self._output_size(self.X)

      a = np.zeros([self.W.shape[0], self.Nout])
      for j in range(self.F):
        for i in range(self.Nout):
          a[j,i] = np.sum(self.X[:, i:i+self.F]*self.W[j])+self.b[j]
      return a

      # 更新
      self = self.optimizer.updatbe(self)
      return self

    # バックプロパゲーション時の処理
    def backward(self, loss):
      db = np.sum(loss)
      dw = np.zeros(self.W.shape)
      dx = np.zeros(self.X.shape)
      for j in range(self.W.shape[0]):
        for i in range(self.W.shape[1]):
          dw[j] += loss[j,i] * self.X[:,i:i+3]

        dx += np.r_[[np.r_[np.sum([loss[i,0]*self.W[i,0,:]], axis=0), 0] + np.r_[0,np.sum([loss[i,1]*self.W[i,0,:]], axis=0)]],
                     [np.r_[np.sum([loss[i,0]*self.W[i,1,:]], axis=0), 0] + np.r_[0,np.sum([loss[i,1]*self.W[i,1,:]], axis=0)]]]
      return db, dw, dx

# 問題2--------------------------------------------------------------
    def _output_size(self, X):
      Nout = int(((X.shape[1]) + (2*self.P) - self.F) / self.S + 1)
      return Nout

In [ ]:
# 全結合層クラス
class FC:
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        self.W = initializer.W(n_nodes1, n_nodes2)
        self.B = initializer.B(n_nodes2)
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する

    # フォワードプロパゲーション時の処理
    def forward(self, X):
      self.X = X
      out = X@self.W+self.B
      return out
    
    # バックプロパゲーション時の処理
    def backward(self, dA):
      self.dZ = dA@(self.W.T)
      self.dW = (self.X.T)@dA
      self.dB = np.sum(dA, axis=0)
      
      # 更新
      self = self.optimizer.update(self)
      return self.dZ

In [ ]:
# 初期化クラス
# SimpleInitializer ------------------------------------------------------------
class SimpleInitializer:
    def __init__(self, sigma):
        self.sigma = sigma

    def W(self, n_nodes1, n_nodes2, filter=None):
      if filter == None:
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
      else:
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2, filter)
      return W

    def B(self, n_nodes2):
      B = self.sigma * np.random.randn(n_nodes2)
      return B

# XavierInitializer ------------------------------------------------------------
class XavierInitializer:
    def __init__(self, sigma):
      self.sigma = sigma/np.sqrt(n_nodes1)

    def W(self, n_nodes1, n_nodes2, filter=None):
      if filter == None:
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
      else:
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2, filter)
      return W

    def B(self, n_nodes2):
      B = self.sigma * np.random.randn(n_nodes2)
      return B

# He-----------------------------------------------------------------------------
class HeInitializer:
    def __init__(self, sigma):
      self.sigma = sigma * np.sqrt(2/n_nodes1) 

    def W(self, n_nodes1, n_nodes2, filter=None):
      if filter == None:
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
      else:
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2, filter)
      return W

    def B(self, n_nodes2):
      B = self.sigma * np.random.randn(n_nodes2)
      return B

In [ ]:
# 最適化クラス
# SGD---------------------------------------------
class SGD:
    def __init__(self, lr):
        self.lr = lr

    def update(self, layer):
      layer.W -= self.lr * layer.dW
      layer.B -= self.lr * layer.dB
      return layer

# AdaGrad----------------------------------------
class AdaGrad:
    def __init__(self, lr):
      self.lr = lr
      self.hw = 0
      self.hb = 0
  
    def update(self, layer):
      self.hw += layer.dW * layer.dW
      self.hb = layer.dB * layer.dB
      layer.W -= self.lr * layer.dW / (np.sqrt(self.hw) +1e-7)
      layer.B -= self.lr * layer.dB / (np.sqrt(self.hb) +1e-7)
      return layer

In [ ]:
# 活性化関数クラス
# ソフトマックス関数のクラス----------------------------------------------
class Softmax:
  def __init__(self):
    pass

  # forward時の処理
  def forward(self, X):
    y_hat = np.exp(X) / np.sum(np.exp(X), axis = 1).reshape(-1,1)
    return y_hat

  # backward時の処理
  def backward(self, X, Y):
    loss = -np.mean(Y * np.log(X + 1e-7))
    dA3 = X - Y
    return dA3, loss

# ReLU関数のクラス----------------------------------------------
class ReLU:
  def __init__(self):
        pass
  # forward時の処理
  def forward(self, X):
    self.X =X
    return np.maximum(0, X)
    
  # backward時の処理
  def backward(self, dout):
    return np.where(self.X > 0, dout, 0)

# tanh関数のクラス----------------------------------------------
class Tanh:
  def __init__(self):
    pass

  # forward時の処理
  def forward(self, X):
    self.out = np.tanh(X)
    return self.out

  # backward時の処理
  def backward(self, X):
    return X*(1-self.out**2)

# sigmoid関数のクラス----------------------------------------------
class Sigmoid:
  def __init__(self):
    pass

  # forward時の処理
  def forward(self, z):
    self.out = 1 / (1+np.exp(-z))
    return self.out

  # backward時の処理
  def backward(self, z):
    return z*(1-self.out)*self.out

### 前処理

In [ ]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 20, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

In [ ]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 平滑化（flatten）
X_train_fltten = X_train.reshape(-1, 784)
X_test_fltten = X_test.reshape(-1, 784)

# float化と0or1処理
X_train_flt = X_train_fltten.astype(np.float)
X_test_flt = X_test_fltten.astype(np.float)
X_train_flt /= 255
X_test_flt /= 255

# one hot処理
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

In [ ]:
# train, testの分割
from sklearn.model_selection import train_test_split
x_train, x_val, y_train_one_hot, y_val_one_hot = train_test_split(np.array(X_train_flt), np.array(y_train_one_hot), test_size=0.2)
print("x_train",x_train.shape, "x_val", x_val.shape, "y_train_one_hot.shape", y_train_one_hot.shape, "y_val_one_hot", y_val_one_hot.shape) # (48000, 784)

In [ ]:
sdnn = ScratchDeepNeuralNetworkClassifier(epoch_num=7)
sdnn.fit(HeInitializer, ReLU, AdaGrad, x_train, y_train_one_hot, x_val, y_val_one_hot, sigma=0.01, lr=0.01)